In [ ]:
# Version 2 - modified by ChatGPT

# pip3 install langchain_ollama

In [23]:
import os, sys

sys.path.append('.')

from langchain_ollama import OllamaLLM
import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

# Load the data from a PDF
file_path = './unix6.pdf'
loader = PyPDFLoader(file_path)
docs = loader.load()

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Define the function to call the Ollama Llama2 model
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    llm = OllamaLLM(model="llama2:70b", base_url="http://localhost:11434")
    
    try:
        response = llm.generate(
            messages=[
                {'role': 'system', 'content': '總是用繁體中文回答！'},
                {'role': 'user', 'content': formatted_prompt}
            ]
        )
        return response['content']
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Define the RAG setup
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Input your question, eg Please summarize in 500 words"),
  outputs="text",
  title="RAG with Llama3",
  description="Ask questions about the provided context",
)

# Launch the Gradio app
iface.launch()
